In [11]:
import sys
import math
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool
import json
from collections import defaultdict
sys.argv=['xx','../data/all_datasets','../www/mysql','../../universal_data/rmsk/rmsk_GRCh38.txt','append']

in_path=sys.argv[1]
out_path=sys.argv[2]
rmsk_f=sys.argv[3]
mode='append' if (len(sys.argv)==5 and sys.argv[4]=='append') else 'overwrite'

# load cell annotation
cell_umaps=[os.path.join(in_path,x) for x in os.listdir(in_path) if x.endswith('.cell_umap.txt')]
cell_umap=pd.concat([pd.read_csv(x,sep='\t',index_col=0) for x in cell_umaps],axis=0)
## replace value of Opc to OPC
cell_umap['predicted.celltype']=cell_umap['predicted.celltype'].replace('Opc','OPC')
for i in range(cell_umap.shape[0]):
    if cell_umap.iloc[i,1] =='Stage_0':
        cell_umap.iloc[i,1]='Control'
    if cell_umap.iloc[i,1] !='Control':
        cell_umap.iloc[i,1]=cell_umap.iloc[i,7].split('_')[0]

# load rmsk
rmsk=pd.read_csv(rmsk_f,sep='\t')
rmsk=rmsk.loc[rmsk['repClass'].isin(['LINE','SINE','LTR']),:]
rmsk['repFamily']=rmsk['repFamily'].apply(lambda x: x.replace('?',''))
families=rmsk['repFamily'].unique()
subfams=rmsk['repName'].unique()

In [12]:
subfam_cell_count=json.load(open(os.path.join(in_path,'subfam_cell_count.json'),'r'))

fam_dic={}
te2family={}
for x in families:
    fam_dic[x]=rmsk.loc[rmsk['repFamily']==x,'repName'].unique().tolist()
    te2family.update({y:x for y in fam_dic[x]})
    
fam_cell_count={x:[] for x in fam_dic}

for x in fam_dic:
    for y in fam_dic[x]:
        if y in subfam_cell_count:
            fam_cell_count[x].extend(subfam_cell_count[y])
for x in fam_dic:
    fam_cell_count[x]=list(set(fam_cell_count[x]))

In [9]:
import pandas as pd

df1=pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})
df2=pd.DataFrame({'b':[1,2,3],'a':[4,5,6]})
pd.concat([df1,df2],axis=0)

,a,b
0,1,4
1,2,5
2,3,6
0,4,1
1,5,2
2,6,3
